In [14]:
from pyspark.sql import SparkSession


In [15]:
spark = SparkSession.builder \
    .appName("datafram_writer_demo") \
    .config("spark.sql.catalogImplementation", "hive") \
    .enableHiveSupport() \
    .getOrCreate() 

In [16]:
spark

In [17]:
orders_schema = "order_id long , order_date string, cust_id long,order_status string"

In [18]:
order_dataframe = spark.read \
.format("csv") \
.schema(orders_schema) \
.load(r"C:\Users\sayed\Desktop\DDP\prac3\data\orders_1gb.csv")

In [19]:
order_dataframe.show()

+--------+--------------------+-------+---------------+
|order_id|          order_date|cust_id|   order_status|
+--------+--------------------+-------+---------------+
|       1|2013-07-25 00:00:...|  11599|         CLOSED|
|       2|2013-07-25 00:00:...|    256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|  12111|       COMPLETE|
|       4|2013-07-25 00:00:...|   8827|         CLOSED|
|       5|2013-07-25 00:00:...|  11318|       COMPLETE|
|       6|2013-07-25 00:00:...|   7130|       COMPLETE|
|       7|2013-07-25 00:00:...|   4530|       COMPLETE|
|       8|2013-07-25 00:00:...|   2911|     PROCESSING|
|       9|2013-07-25 00:00:...|   5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:...|   5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:...|    918| PAYMENT_REVIEW|
|      12|2013-07-25 00:00:...|   1837|         CLOSED|
|      13|2013-07-25 00:00:...|   9149|PENDING_PAYMENT|
|      14|2013-07-25 00:00:...|   9842|     PROCESSING|
|      15|2013-07-25 00:00:...|   2568|       CO

In [20]:
order_dataframe = spark.read \
.format("csv") \
.schema(orders_schema) \
.load("C:\\Users\\sayed\\Desktop\\DDP\\prac3\\data\\sparkwriterdemo1")

In [21]:
order_dataframe.createOrReplaceTempView("orders")

In [22]:
spark.sql("select count(*) from orders where order_status = 'CLOSED'").show()

+--------+
|count(1)|
+--------+
| 2833500|
+--------+



In [23]:
order_dataframe.write \
.format("csv")\
.mode("overwrite") \
.partitionBy("order_status") \
.option("path", "C:\\Users\\sayed\\Desktop\\DDP\\prac3\\data\\partitiondemo1") \
.save()

In [24]:
order_dataframe.rdd.getNumPartitions()  # type: ignore 

9

In [ ]:
spark.sql("select distinct(order_status) from orders").count()

9

In [ ]:
orders_df = spark.read \
.format("csv") \
.schema(orders_schema) \
.load("C:\\Users\\sayed\\Desktop\\DDP\\prac3\\data\\partitiondemo1")



In [27]:
order_dataframe.createOrReplaceTempView("orders")

In [28]:
spark.sql("select count(*) from orders where order_status = 'CLOSED'").show()

+--------+
|count(1)|
+--------+
| 2833500|
+--------+



In [29]:
spark.sql("select count(*) from orders where cust_id = 11599").show()

+--------+
|count(1)|
+--------+
|    1875|
+--------+



In [30]:
spark.sql("select count(*) from orders where cust_id = 11599 and order_status = 'CLOSED'").show()

+--------+
|count(1)|
+--------+
|     375|
+--------+



In [31]:
customers_dataframe = spark.read \
.format("csv") \
.option("inferSchema", True) \
.load(r"C:\\Users\\sayed\\Desktop\\DDP\\prac2\\data\\part-00000")

In [34]:
print(customers_dataframe.columns)


['_c0', '_c1', '_c2', '_c3']


In [35]:
customers_final_dataframe = customers_dataframe.toDF(
    "customer_id", "customer_fname", "customer_lname", "customer_email"
)


In [37]:
from pyspark.sql.functions import lit

# Adding placeholder columns with default values (None or empty strings)
customers_dataframe = customers_dataframe \
    .withColumn("customer_password", lit(None)) \
    .withColumn("customer_street", lit(None)) \
    .withColumn("customer_city", lit(None)) \
    .withColumn("customer_state", lit(None)) \
    .withColumn("customer_zipcode", lit(None))

customers_final_dataframe = customers_dataframe.toDF(
    "customer_id", "customer_fname", "customer_lname", "customer_email", 
    "customer_password", "customer_street", "customer_city", "customer_state", "customer_zipcode"
)
